In [1]:
#IMPORTANDO BIBLIOTECAS
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import sqlite3
import time
from tqdm import tqdm
import os
import random

In [2]:
#GUARDANDO LISTAS
Links_Americanas = []
Urls_Americanas = []
Sellers_Americanas = []
Country_Americanas = []
Price_Americanas = []
SKU_Americanas = []
Title_Americanas = []
Installment_Americanas_quantidade = []
Installment_Americanas_valor_parcela = []
Installment_Americanas_valor_total = []
More_offers_americanas = []

In [3]:
#Configurando Header
headers = {'authority':'www.americanas.com.br','scheme':'https','accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
           'accept-encoding':'gzip, deflate, br','referer':'https://www.americanas.com.br/', 'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36 OPR/85.0.4341.60'}

In [4]:
User_agents_list = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36 OPR/85.0.4341.60',
                    'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.130 Safari/537.36',
                    'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36',
                    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.130 Safari/537.36',
                    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.101 Safari/537.36']

In [5]:
def Random_User_Agents():
    Choice_User_Agent = random.choice(User_agents_list)

    #Configurando Header
    headers = {'authority':'www.americanas.com.br','scheme':'https','accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
               'accept-encoding':'gzip, deflate, br','referer':'https://www.americanas.com.br/', 'user-agent':Choice_User_Agent}


    return headers

In [6]:
def getting_n_creating_americanas_urls(brand):

    Database_path = r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1" + "\Data\\" + brand + "\\" + brand + ".db"

    table = brand + "_products"

    #Criando a Query
    query = "SELECT * FROM " + table

    #Entrando dentro do databse
    connection = sqlite3.connect(Database_path)

    #Criando o dataset em brando
    df = pd.read_sql_query(query, connection)

    #Passando todo o Dataframe para LowerCase
    df = df.apply(lambda x: x.astype(str).str.lower())

    #Arrumando espaços vazios
    # Arrumano os espaços vazios
    df['Product_Name'] = df['Product_Name'].str.replace(" ", "-")

    # Criando uma nova coluna no database com a formatação certa
    df['Urls'] = df['Brand'] + "-" + df['Product_Name']

    # Criando a nova coluna que são as urls de pesquisa
    df['Urls_search'] = "https://www.americanas.com.br/busca/" + df['Urls']

    return df

In [7]:
Df_Products_Links = getting_n_creating_americanas_urls("Wacom")

In [8]:
def search_links(url):
    global Links_Americanas

    time.sleep(10)

    Headers_Choice = Random_User_Agents()

    response = requests.get(url, headers=Headers_Choice)
    
    if response.status_code != 200:
        Headers_Choice = Random_User_Agents()
        response = requests.get(url, headers=Headers_Choice)

    else:
        print(response.status_code)

    html = response.text

    bs = BeautifulSoup(html, 'html.parser')

    for link in bs.find_all("a", href=True):
        Links_Americanas.append("https://www.americanas.com.br" + link['href'])

    Links_Americanas = [s for s in Links_Americanas if 'produto' in s]

In [9]:
for url in tqdm(Df_Products_Links['Urls_search']):
    search_links(url)

 10%|█         | 1/10 [00:10<01:38, 10.90s/it]

200


 20%|██        | 2/10 [00:21<01:26, 10.82s/it]

200


 30%|███       | 3/10 [00:32<01:15, 10.80s/it]

...

200


 90%|█████████ | 9/10 [01:36<00:10, 10.65s/it]

200


100%|██████████| 10/10 [01:47<00:00, 10.71s/it]

200


In [10]:
len(Links_Americanas)

109

In [11]:
def get_atributes(url):
    time.sleep(30)

    Headers_Choice = Random_User_Agents()

    response = requests.get(url, headers=Headers_Choice)

    if response.status_code != 200:
        
        Headers_Choice = Random_User_Agents()
        response = requests.get(url, headers=Headers_Choice)

    else:
        pass

    html = response.text

    soup = BeautifulSoup(html, 'html.parser')

    #Title
    try:
        Title_Americanas.append(soup.find(class_='product-title__Title-sc-1hlrxcw-0 jyetLr').text)
    except:
        Title_Americanas.append("Erro")

    #Preço
    try:
        Price_Americanas.append(soup.find(class_='styles__PriceText-sc-x06r9i-0 dUTOlD priceSales').text)
    except:
        Price_Americanas.append('Erro')

    #Installment
    try:
        Installment_Americanas_valor_parcela.append(soup.find(class_='payment-installment-text__Text-sc-12txe9z-0 bfFyfi').text)
    except:
        Installment_Americanas_valor_parcela.append("Erro")

    #Seller
    try:
        Sellers_Americanas.append(soup.find(class_='offers-box__Wrapper-sc-189v1x3-0 kegaFO').text)
    except:
        Sellers_Americanas.append("Erro")

    #More offers 
    try:
        More_offers_americanas.append(soup.find(class_='more-offers__Touchable-sc-15yqej3-2 hYfNEd').text)
    except:
        More_offers_americanas.append("Erro")

In [12]:
for url in tqdm(Links_Americanas):
    get_atributes(url)

100%|██████████| 109/109 [55:40<00:00, 30.64s/it]


In [13]:
#GUARDANDO LISTAS
print(len(Links_Americanas))
print(len(Urls_Americanas))
print(len(Sellers_Americanas))
print(len(Country_Americanas))
print(len(Price_Americanas))
print(len(SKU_Americanas))
print(len(Title_Americanas))
print(len(Installment_Americanas_quantidade))
print(len(Installment_Americanas_valor_parcela))
print(len(Installment_Americanas_valor_total))
print(len(More_offers_americanas))

109
0
109
0
109
0
109
0
109
0
109


In [68]:
def create_dataframe(url, sellers, price, installment, title):
    df_raw = pd.DataFrame()

    Hoje = pd.to_datetime('today', errors='ignore').date()

    df_raw['URL'] = url

    df_raw['DATE'] = Hoje

    df_raw['MARKETPLACE'] = 'Americanas'

    df_raw['SELLER'] = sellers
    df_raw['SELLER'] = df_raw['SELLER'].str.replace("Este produto é vendido por","")
    df_raw['SELLER'] = df_raw['SELLER'].str.partition(" e")[0]
    


    df_raw['PRICE'] = price
    df_raw['PRICE'] = df_raw['PRICE'].str.replace("R$ ", "", regex=False)
    df_raw['PRICE'] = df_raw['PRICE'].str.replace(".","")
    df_raw['PRICE'] = df_raw['PRICE'].str.replace(",",".")



    df_raw['Installment_full'] = installment
    df_raw['PARCEL'] = df_raw['Installment_full'].str.partition('x')[0].str.partition("até ")[2]

    df_raw['INSTALLMENT'] = df_raw['Installment_full'].str.partition("x")[2].str.partition("R$ ")[2]
    df_raw['INSTALLMENT'] = df_raw['INSTALLMENT'].str.replace(",",".")

    df_raw['PRODUCT'] = title

    df_raw['MORE'] = More_offers_americanas

    df_raw['ID'] = df_raw['URL'].str.partition("produto/")[2].str.partition('?')[0]

    df_raw = df_raw[['DATE', 'URL', 'MARKETPLACE', 'SELLER', 'PRICE', 'PARCEL', 'INSTALLMENT', 'ID', 'PRODUCT',  'MORE']]


    return df_raw

In [69]:
Dataset_Americanas = create_dataframe(Links_Americanas, Sellers_Americanas, Price_Americanas, Installment_Americanas_valor_parcela, Title_Americanas)

C:\Users\pedro\AppData\Local\Temp\ipykernel_14720\3503551429.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_raw['PRICE'] = df_raw['PRICE'].str.replace(".","")


In [70]:
Dataset_Americanas

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,ID,PRODUCT,MORE
0,2022-05-18,https://www.americanas.com.br/produto/13371036...,Americanas,FiberMega,197.91,4,56.22,133710364,Mesa Digitalizadora One By Wacom CTL472,27 outras ofertas deste mesmo produto
1,2022-05-18,https://www.americanas.com.br/produto/37195263...,Americanas,SHOPAR,277.97,5,59.78,3719526302,Mesa Digitalizadora Wacom One CTL472 Pequena B...,Erro
2,2022-05-18,https://www.americanas.com.br/produto/40663058...,Americanas,HD Store,305.85,6,54.81,4066305835,Mesa Digitalizadora Wacom Ctl472 One,Erro
3,2022-05-18,https://www.americanas.com.br/produto/38397139...,Americanas,Catarinas,598.92,8,80.50,3839713911,Mesa Digitalizadora CTL472 One Wacom,Erro
4,2022-05-18,https://www.americanas.com.br/produto/49586272...,Americanas,RL2INFO SOLUÇÕES EM TI,371.99,8,49.99,4958627240,Mesa Digitalizadora Wacom One By Pequena - Ctl472,Erro
...,...,...,...,...,...,...,...,...,...,...
104,2022-05-18,https://www.americanas.com.br/produto/48009631...,Americanas,Haylou,3378.82,8,469.28,4800963147,Mesa Digitalizadora Wacom Intuos Pro PTH660 co...,Erro
105,2022-05-18,https://www.americanas.com.br/produto/61058374...,Americanas,Tendinfo,6312.84,8,848.50,61058374,Mesa Digitalizadora Wacom Intuos Pro Média (pt...,Erro
106,2022-05-18,https://www.americanas.com.br/produto/48387443...,Americanas,MultiShop,4290.57,8,536.32,4838744302,Mesa Digitalizadora Wacom Intuos Pro PTH660 co...,Erro
107,2022-05-18,https://www.americanas.com.br/produto/23011597...,Americanas,MIRANDA,2037.91,8,254.73,2301159787,Mesa Digitalizadora Intuos Bluetooth Média - V...,Erro


In [71]:
Dataset_Americanas.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Americanas_Wacom.xlsx", index=False)